<a href="https://colab.research.google.com/github/techshot25/MachineLearningPractice/blob/master/ScrappyDecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Making a decision tree

This is my attempt to design a decision tree from scratch.

In [0]:
from __future__ import print_function

In [0]:
# let's make some training data where all but the last column are features 
# and the last column is the classifier

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

# making labels for each column
header = ['Color' , 'diameter' , 'label']

In [4]:
# this returns the unique values for each column
# just excludes repetitions
# this displays uniqueness in the datasets

def unique_vals(rows, col):
  return set([row[col] for row in rows])

print(unique_vals(training_data, 0))
print(unique_vals(training_data, 1))

{'Yellow', 'Green', 'Red'}
{1, 3}


In [5]:
# this displays how many of each classifier are there

def class_counts(rows):
  counts = {} # start a dictionary
  for row in rows:
    label = row[-1]
    if label not in counts:
      counts[label] = 0
    counts[label] += 1
  return counts

class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [6]:
# checking if the data is numeric

def is_numeric(value):
  return isinstance(value, int) or isinstance(value, float)

is_numeric(training_data[0][1])

True

In [0]:
# this is where the fun begins

class Question:
  def __init__(self,column,value):
    self.column = column
    self.value = value
  
  # type of question to ask:
  # if numeric, then an inequality is possible
  # else it must be an exact match
  
  def match(self, example):
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value
  def __repr__(self):
    condition = "=="
    if is_numeric(self.value):
      condition = ">="
    return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [8]:
# this simply returns true to check if our code works
# it sees if the first feature is green, which is true

q = Question(0,'Green')

example = training_data[0]
q.match(example)

True

In [0]:
# next is to develop a partition for our dataset

def partition(rows, question):
  true_rows, false_rows =  [], []
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows, false_rows

In [10]:
# checking our first tree branch with a true and false question:

true_rows, false_rows = partition(training_data, Question(0, 'Red'))

print(true_rows)

print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [13]:
#using impurities to find certainty between 0 and 1
#lower impurity means more confidence and lower uncertainty

def gini(rows):
  counts = class_counts(rows)
  impurity = 1
  for lbl in counts:
    prob_of_lbl = counts[lbl] / float(len(rows))
    impurity -= prob_of_lbl**2
  return impurity

#another way to think of this is the estimate for confidence intervals from p&s

no_mixing = [
    ['Apple'],['Apple']
]

lots_of_mixing = [
    ['Apple'],['Orange'],['Grape'],['Blueberry'],['Blackberry']
]

print(gini(no_mixing))
print(gini(lots_of_mixing))

0.0
0.7999999999999998


In [14]:
# gaining information allows us to take the next step, now we can move on

def info_gain(left, right, current_uncertainty):
  p = float(len(left)) / (len(left)+len(right))
  return current_uncertainty - p * gini(left) - (1-p)* gini(right)

current_uncertainty = gini(training_data)

print(current_uncertainty)

0.6399999999999999


In [15]:
# how much info did we gain by partitioning "Green"?

true_rows, false_rows = partition(training_data, Question(0,'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [16]:
# how much info did we gain by partitioning "red"?

true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [17]:
# we gained more info by partitioning red.
# to illustrate, let's split into true and false with this partition

print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


In [0]:
# this filtering process is working well so far

In [23]:
# let's find the method that gives us the best partition we can perform
# the one that will give us more information

def find_best_split(rows):
  best_gain = 0 #starting with zero
  best_question = None
  current_uncertainty = gini(rows)
  n_features = len(rows[0]) - 1 #number of columns with features
  for col in range(n_features): #best question for each feature
    values = set([row[col] for row in rows]) #unique values in column
    
    for val in values:
      
      question = Question(col, val)
    
      true_rows, false_rows = partition(rows, question)
    
      # if there is no dataset division then move on
      if len(true_rows) == 0 or len(false_rows) == 0:
        continue
    
      gain = info_gain(true_rows, false_rows, current_uncertainty)
    
      if gain >= best_gain:
         best_gain, best_question = gain, question
      
  return best_gain, best_question

# we now have a method to get the best question for each feature

best_gain, best_question = find_best_split(training_data)

print(best_question)

Is diameter >= 3?


In [0]:
# now that we have the best value for each question
# we can now classify our data into leaves from the branch

class Leaf:
  def __init__(self,rows):
    self.predictions = class_counts(rows)

class Decision_Node: #making nodes for each question to hold a reference
  def __init__(self, question, true_branch, false_branch):
    self.question = question
    self.true_branch = true_branch
    self.false_branch = false_branch

def build_tree(rows): #using a recursion method to find each case
  gain, question = find_best_split(rows)
  
  if gain == 0:
    return Leaf(rows)
  
  true_rows, false_rows = partition(rows, question)
  
  true_branch = build_tree(true_rows)
  
  false_branch = build_tree(false_rows)
  
  return Decision_Node(question, true_branch, false_branch)

In [0]:
def print_tree(node, spacing=""):
  # base case: we reached a leaf
  if isinstance(node,Leaf):
    print(spacing + 'Predict', node.predictions)
    return
  # print the question here
  print(spacing + str(node.question))
  
  # call itself for the true branch case
  print(spacing + '--> True:')
  print_tree(node.true_branch, spacing + "  ")
  
  # call itself for the false branch case
  print(spacing + '--> False:')
  print_tree(node.false_branch, spacing + "  ")

In [30]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is Color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [31]:
# setting the rules for recursion

def classify(row, node):
  if isinstance(node, Leaf):
    return node.predictions
  
  #decide whether we follow true or false branch
  
  if node.question.match(row):
    return classify(row, node.true_branch)
  else:
    return classify(row, node.false_branch)
  
classify(training_data[0], my_tree)

{'Apple': 1}

In [32]:
def print_leaf(counts):
  total = sum(counts.values()) * 1.0
  probs = {}
  for lbl in counts.keys():
    probs[lbl] = str(int(counts[lbl] / total * 100)) + '%'
  return probs

print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [33]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [35]:
#testing prediction accuracy of our decision tree

testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

for row in testing_data:
  print('Actual %s. Predicted %s' % 
        (row[-1],print_leaf(classify(row,my_tree))))

Actual Apple. Predicted {'Apple': '100%'}
Actual Apple. Predicted {'Apple': '50%', 'Lemon': '50%'}
Actual Grape. Predicted {'Grape': '100%'}
Actual Lemon. Predicted {'Apple': '50%', 'Lemon': '50%'}
